In [35]:
from transformers import AutoTokenizer, DataCollatorWithPadding, AutoModelForSequenceClassification, TrainingArguments, Trainer, TextClassificationPipeline
from datasets import Dataset, ClassLabel, Features, Value, load_dataset
import evaluate
import pandas as pd
import numpy as np
import shutil
import os
from scipy.special import softmax
from sklearn.metrics import (
    confusion_matrix,
    cohen_kappa_score,
    classification_report,
    accuracy_score,
    roc_auc_score,
)

In [36]:
tokenizer = AutoTokenizer.from_pretrained("PlanTL-GOB-ES/roberta-base-bne")

In [37]:
train = pd.read_csv("data/processed/train.csv")
test = pd.read_csv("data/processed/test.csv")
dev = pd.read_csv("data/processed/dev.csv")
train.label = train.label.map({'normal':0, 'incivilidad': 1, 'odio':2})
test.label = test.label.map({'normal':0, 'incivilidad': 1, 'odio':2})
dev.label = dev.label.map({'normal':0, 'incivilidad': 1, 'odio':2})

In [38]:
features = Features({"text": Value("string"), "label": ClassLabel(num_classes=3, names=['normal','incivilidad','odio'])})

In [39]:
train_ds = Dataset.from_pandas(train, split="train")
test_ds = Dataset.from_pandas(test, split="test")
dev_ds = Dataset.from_pandas(dev, split="dev")

In [40]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

In [41]:
tokenized_train_ds = train_ds.map(preprocess_function, batched=True)
tokenized_test_ds = test_ds.map(preprocess_function, batched=True)
tokenized_dev_ds = dev_ds.map(preprocess_function, batched=True)

Map:   0%|          | 0/8549 [00:00<?, ? examples/s]

Map:   0%|          | 0/2455 [00:00<?, ? examples/s]

Map:   0%|          | 0/1210 [00:00<?, ? examples/s]

In [42]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [43]:
auc = evaluate.load("roc_auc", "multiclass")

In [14]:
def auc_score(test_set, predicted_set):
    high_predicted = np.array([prediction[2] for prediction in predicted_set])
    medium_predicted = np.array(
        [prediction[1] for prediction in predicted_set]
    )
    low_predicted = np.array([prediction[0] for prediction in predicted_set])
    inc_test = np.where(test_set == "incivilidad", 1.0, 0.0)
    odio_test = np.where(test_set == "odio", 1.0, 0.0)
    normal_test = np.where(test_set == "normal", 1.0, 0.0)
    auc_high = roc_auc_score(inc_test, high_predicted)
    auc_med = roc_auc_score(odio_test, medium_predicted)
    auc_low = roc_auc_score(normal_test, low_predicted)
    auc_w = (
        normal_test.sum() * auc_low
        + odio_test.sum() * auc_med
        + inc_test.sum() * auc_high
    ) / (normal_test.sum() + odio_test.sum() + inc_test.sum())
    return auc_w


def evaluate(predicted_probabilities, y_test, labels):
    # Importante: al transformar los arreglos de probabilidad a clases,
    # entregar el arreglo de clases aprendido por el clasificador.
    # (que comunmente, es distinto a ['normal', 'odio', 'incivilidad'])
    predicted_labels = [
        labels[np.argmax(item)] for item in predicted_probabilities
    ]

    print("Matriz de confusión")
    print(
        confusion_matrix(
            y_test, predicted_labels, labels=["normal", "odio", "incivilidad"]
        )
    )

    print("\nReporte de clasificación:\n")
    print(
        classification_report(
            y_test, predicted_labels, labels=["normal", "odio", "incivilidad"]
        )
    )
    # Reorder predicted probabilities array.
    labels = labels.tolist()

    predicted_probabilities = predicted_probabilities[
        :,
        [
            labels.index("normal"),
            labels.index("odio"),
            labels.index("incivilidad"),
        ],
    ]

    auc = round(auc_score(y_test, predicted_probabilities), 3)
    print("Métricas:\n\nAUC: ", auc, end="\t")
    kappa = round(cohen_kappa_score(y_test, predicted_labels), 3)
    print("Kappa:", kappa, end="\t")
    accuracy = round(accuracy_score(y_test, predicted_labels), 3)
    print("Accuracy:", accuracy)
    print("------------------------------------------------------\n")
    return np.array([auc, kappa, accuracy])


In [44]:
id2label = {0: "normal", 1: "incivilidad", 2:'odio'}
label2id = {"normal": 0, "incivilidad": 1, 'odio':3}

In [45]:
def compute_metrics(eval_pred):

    predictions, labels = eval_pred
    # labels = np.array(list(map(lambda x: id2label[x],labels)))

    # predictions = np.argmax(predictions, axis=1)
    predictions = softmax(predictions, axis=1)

    # return accuracy.compute(predictions=predictions, references=labels)
    # auc = auc_score(labels, predictions)
    # return {"AUC": auc}

    return auc.compute(prediction_scores=predictions, references=labels, multi_class='ovr', average='weighted')

In [46]:
model = AutoModelForSequenceClassification.from_pretrained(
    "PlanTL-GOB-ES/roberta-base-bne", num_labels=3, id2label=id2label, label2id=label2id
)

Some weights of the model checkpoint at PlanTL-GOB-ES/roberta-base-bne were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.decoder.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at PlanTL-GOB-ES/roberta-base-bne and are newly initialized: ['classifier.out_proj.weight', 'classi

In [18]:
def model_init(trial):
    return AutoModelForSequenceClassification.from_pretrained(
    "PlanTL-GOB-ES/roberta-base-bne", num_labels=3, id2label=id2label, label2id=label2id
)

In [19]:
training_args = TrainingArguments(
    output_dir="hater_model",
    learning_rate=3.3681453220115495e-05,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.04142006478907374,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

In [20]:
trainer = Trainer(
    model=None,
    model_init=model_init,
    args=training_args,
    train_dataset=tokenized_train_ds,
    eval_dataset=tokenized_dev_ds,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

Some weights of the model checkpoint at PlanTL-GOB-ES/roberta-base-bne were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.weight', 'lm_head.decoder.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at PlanTL-GOB-ES/roberta-base-bne and are newly initialized: ['classifier.dense.bias', 'classifier.

In [ ]:
def optuna_hp_space(trial):
    return {
        "learning_rate": trial.suggest_float("learning_rate", 2e-6, 2e-4, log=True),
        "weight_decay": trial.suggest_float("weight_decay", 0.01, 0.1)
    }

In [22]:
best_trial = trainer.hyperparameter_search(
    direction="maximize",
    backend="optuna",
    hp_space=optuna_hp_space,
    n_trials=20,
)

[I 2023-04-22 21:05:33,148] A new study created in memory with name: no-name-77bfafa9-54ce-4d58-aad2-fd02c92d340c
Some weights of the model checkpoint at PlanTL-GOB-ES/roberta-base-bne were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.decoder.weight', 'lm_head.decoder.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the mo

Epoch,Training Loss,Validation Loss,Roc Auc
1,0.872500,0.675625,0.873060
2,0.569400,0.574227,0.906189
3,0.453600,0.551315,0.916141
4,0.389500,0.556723,0.917427
5,0.336400,0.561393,0.917600


[I 2023-04-22 21:19:43,623] Trial 0 finished with value: 0.9176002067742549 and parameters: {'learning_rate': 2.4117335463139266e-06, 'weight_decay': 0.035010786196879945}. Best is trial 0 with value: 0.9176002067742549.
Some weights of the model checkpoint at PlanTL-GOB-ES/roberta-base-bne were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.decoder.weight', 'lm_head.decoder.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSeque

Epoch,Training Loss,Validation Loss,Roc Auc
1,0.981500,1.054146,0.488433
2,1.054300,1.054010,0.413259
3,1.056200,1.053369,0.543055
4,1.052500,1.053610,0.644266
5,1.053800,1.053360,0.554773


[I 2023-04-22 21:32:39,243] Trial 1 finished with value: 0.5547728186732589 and parameters: {'learning_rate': 0.00016557987290905573, 'weight_decay': 0.055034838492446336}. Best is trial 0 with value: 0.9176002067742549.
Some weights of the model checkpoint at PlanTL-GOB-ES/roberta-base-bne were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.decoder.weight', 'lm_head.decoder.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSeque

...

[I 2023-04-23 00:06:17,263] Trial 17 pruned. 
Some weights of the model checkpoint at PlanTL-GOB-ES/roberta-base-bne were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.decoder.weight', 'lm_head.decoder.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at PlanTL-GOB-ES/roberta-base-bne and are newly initi

Epoch,Training Loss,Validation Loss,Roc Auc
1,0.579800,0.473729,0.937188
2,0.271400,0.488919,0.944637
3,0.113900,0.807505,0.938940
4,0.037400,0.875180,0.940500
5,0.015000,0.904226,0.939423


[I 2023-04-23 00:18:47,556] Trial 18 finished with value: 0.9394228014591126 and parameters: {'learning_rate': 1.821393890976017e-05, 'weight_decay': 0.0889114793394266}. Best is trial 11 with value: 0.9424311353875653.
Some weights of the model checkpoint at PlanTL-GOB-ES/roberta-base-bne were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.decoder.weight', 'lm_head.decoder.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequen

Epoch,Training Loss,Validation Loss,Roc Auc
1,0.602000,0.468934,0.936398
2,0.295900,0.488706,0.943949
3,0.141300,0.752380,0.937455
4,0.054000,0.848251,0.938572
5,0.022700,0.896421,0.938066


[I 2023-04-23 00:31:17,783] Trial 19 finished with value: 0.93806636109233 and parameters: {'learning_rate': 1.2201115460075705e-05, 'weight_decay': 0.07501041118964788}. Best is trial 11 with value: 0.9424311353875653.


In [23]:
best_trial

BestRun(run_id='11', objective=0.9424311353875653, hyperparameters={'learning_rate': 2.0400185548331594e-05, 'weight_decay': 0.09633315261540512}, run_summary=None)

In [18]:
training_args2 = TrainingArguments(
    output_dir="hater_model",
    learning_rate=2.0400185548331594e-05,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.09633315261540512,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

In [19]:
trainer2 = Trainer(
    model=model,
    args=training_args2,
    train_dataset=tokenized_train_ds,
    eval_dataset=tokenized_dev_ds,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [20]:
trainer2.train()

/opt/conda/lib/python3.9/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Roc Auc
1,0.574400,0.485195,0.940826
2,0.263200,0.515967,0.944641
3,0.111000,0.818226,0.939340
4,0.038900,0.868572,0.944426
5,0.010300,0.910280,0.943993


TrainOutput(global_step=2675, training_loss=0.18711335186646363, metrics={'train_runtime': 731.8263, 'train_samples_per_second': 58.409, 'train_steps_per_second': 3.655, 'total_flos': 2041820247096378.0, 'train_loss': 0.18711335186646363, 'epoch': 5.0})

In [21]:
pipe = TextClassificationPipeline(model=model, tokenizer=tokenizer, return_all_scores=True, device=0)

/opt/conda/lib/python3.9/site-packages/transformers/pipelines/text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar funcionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [22]:
predicted = pipe(test_ds['text'])

In [23]:
predicted_probabilities = np.array([[p[0]['score'],p[1]['score'],p[2]['score']] for p in predicted])

In [24]:
learned_labels = np.array(['normal','incivilidad','odio'])

In [25]:
y_test = np.array(list(map(lambda x: id2label[x],test_ds['label'])))

In [26]:
scores = evaluate(predicted_probabilities, y_test, learned_labels)

Matriz de confusión
[[773  15  72]
 [132 315  57]
 [106  37 948]]

Reporte de clasificación:

              precision    recall  f1-score   support

      normal       0.76      0.90      0.83       860
        odio       0.86      0.62      0.72       504
 incivilidad       0.88      0.87      0.87      1091

    accuracy                           0.83      2455
   macro avg       0.83      0.80      0.81      2455
weighted avg       0.84      0.83      0.83      2455

Métricas:

AUC:  0.945	Kappa: 0.729	Accuracy: 0.829
------------------------------------------------------



In [27]:
target = pd.read_csv("https://raw.githubusercontent.com/dccuchile/CC6205/master/assignments/new/assignment_1/target/target.tsv", sep="\t")

In [29]:
predictions = pipe(list(target.texto))

In [30]:
predictions_df = [(p[0]['score'],p[1]['score'],p[2]['score']) for p in predictions]
predictions_df = pd.DataFrame(
    predictions_df,
    columns=['normal','incivilidad','odio']
)

In [31]:
predictions_df['id'] = target.id.values

In [32]:
predictions_df = predictions_df[["id", "normal", "odio", "incivilidad"]]

In [34]:
predicted_target = {}

# Crear carpeta ./predictions
if not os.path.exists("./predictions"):
    os.mkdir("./predictions")

else:
    # Eliminar predicciones anteriores:
    shutil.rmtree("./predictions")
    os.mkdir("./predictions")


# Guardar predicciones en archivos separados.
predictions_df.to_csv(
    "./predictions/prediction.txt", sep="\t", header=False, index=False
)

# Crear archivo zip
a = shutil.make_archive("predictions", "zip", "./predictions")